## Домашнее задание №6

In [48]:
import pandas as pd
import numpy as np
data = pd.read_csv("Maternal_Health.csv", header=None)
data

,0,1,2,3,4,5,6
0,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
1,25,130,80,15,98,86,high risk
2,35,140,90,13,98,70,high risk
3,29,90,70,8,100,80,high risk
4,30,140,85,7,98,70,high risk
...,...,...,...,...,...,...,...
1010,22,120,60,15,98,80,high risk
1011,55,120,90,18,98,60,high risk
1012,35,85,60,19,98,86,high risk
1013,43,120,90,18,98,70,high risk


#### Обозначения:
1. Возраст: любой возраст в годах, когда женщина беременна.
2. Систолическое АД: Верхнее значение артериального давления в мм рт. ст., еще один важный атрибут во время беременности.
3. Диастолическое АД: более низкое значение артериального давления в мм рт. ст., еще один важный атрибут во время беременности.
4. BS: Уровень глюкозы в крови выражен в молярной концентрации, ммоль/л.
5. BodyTemp: Температура тела, градусов F.
6. HeartRate: нормальная частота сердечных сокращений в состоянии покоя в ударах в минуту.
7. Уровень риска: прогнозируемый уровень интенсивности риска во время беременности с учетом предыдущего атрибута

Удалим 0 строку из датасета

In [49]:
data.drop(labels=[0], inplace=True)
data.head()

,0,1,2,3,4,5,6
1,25,130,80,15,98,86,high risk
2,35,140,90,13,98,70,high risk
3,29,90,70,8,100,80,high risk
4,30,140,85,7,98,70,high risk
5,35,120,60,6.1,98,76,low risk


Посмотрим на количество объектов в датасете и количество объектов с высоким риском

In [50]:
print(f"Total number: {data.shape[0]}")
print(f"High-risk quantity: {len(data[data[6] == 'high risk'])}")

Total number: 1014
High-risk quantity: 272


Давайте разметим наш датасет (женщины с высоким риском - метка 1) - это класс 1. 

Мы предполагаем, что люди со схожими на класс 1 признаками заболеваний, также могут быть в зоне риска. 

In [51]:
data['y'] = data[6].apply(lambda x: 1 if x == 'high risk' else 0)
data.tail()

,0,1,2,3,4,5,6,y
1010,22,120,60,15,98,80,high risk,1
1011,55,120,90,18,98,60,high risk,1
1012,35,85,60,19,98,86,high risk,1
1013,43,120,90,18,98,70,high risk,1
1014,32,120,65,6,101,76,mid risk,0


Удалим 6 столбец из датасета

In [52]:
data.drop(columns=[6], inplace=True)
data.head()

,0,1,2,3,4,5,y
1,25,130,80,15,98,86,1
2,35,140,90,13,98,70,1
3,29,90,70,8,100,80,1
4,30,140,85,7,98,70,1
5,35,120,60,6.1,98,76,0


In [55]:
data = data.astype(float)
data

,0,1,2,3,4,5,y
1,25.0,130.0,80.0,15.0,98.0,86.0,1.0
2,35.0,140.0,90.0,13.0,98.0,70.0,1.0
3,29.0,90.0,70.0,8.0,100.0,80.0,1.0
4,30.0,140.0,85.0,7.0,98.0,70.0,1.0
5,35.0,120.0,60.0,6.1,98.0,76.0,0.0
...,...,...,...,...,...,...,...
1010,22.0,120.0,60.0,15.0,98.0,80.0,1.0
1011,55.0,120.0,90.0,18.0,98.0,60.0,1.0
1012,35.0,85.0,60.0,19.0,98.0,86.0,1.0
1013,43.0,120.0,90.0,18.0,98.0,70.0,1.0


Посмотрим на соотношение классов (P vs U), где P - позитивы и U - все остальные неразмеченные данные

In [56]:
data['y'].value_counts()

0.0    742
1.0    272
Name: y, dtype: int64

Разбиваем выборку на тренировочную и тестовую части и обучаем модель

In [57]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [18]:
!pip install xgboost

In [58]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверяем качество

In [60]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print(f"f1: {f1 * 100.0:.2f}%") 
    rec = recall_score(y_test, y_predict, average='binary')
    print(f"recall: {rec * 100.0:.2f}%") 
    prc = precision_score(y_test, y_predict, average='binary')
    print(f"precision: {prc * 100.0:.2f}%" ) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 90.91%
recall: 87.72%
precision: 94.34%


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [102]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[1:pos_sample_len]

Using 68/272 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [103]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    947
 1     67
Name: class_test, dtype: int64


* 67 позитивных примеров (1)
* 947 без разметки (-1)

In [75]:
mod_data.head(10)

,0,1,2,3,4,5,y,class_test
1,25.0,130.0,80.0,15.00,98.0,86.0,1.0,1
2,35.0,140.0,90.0,13.00,98.0,70.0,1.0,-1
3,29.0,90.0,70.0,8.00,100.0,80.0,1.0,-1
4,30.0,140.0,85.0,7.00,98.0,70.0,1.0,-1
5,35.0,120.0,60.0,6.10,98.0,76.0,0.0,1
6,23.0,140.0,80.0,7.01,98.0,70.0,1.0,-1
7,23.0,130.0,70.0,7.01,98.0,78.0,0.0,-1
8,35.0,85.0,60.0,11.00,102.0,86.0,1.0,-1
9,32.0,120.0,90.0,6.90,98.0,70.0,0.0,-1
10,42.0,130.0,80.0,18.00,98.0,70.0,1.0,-1


#### Random negative sampling

In [104]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(67, 8) (67, 8)


In [105]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 78.35%
recall: 88.37%
precision: 70.37%


Составим таблицу с метриками:

In [106]:
models_score = [["Классфификатор xgboost", 90.91, 87.72, 94.34],
                ["random negative sampling", 78.35, 88.37, 70.37]]

In [107]:
pd.DataFrame(models_score, columns=["Модель", "f1 score","recall", "precision"])

,Модель,f1 score,recall,precision
0,Классфификатор xgboost,90.91,87.72,94.34
1,random negative sampling,78.35,88.37,70.37


Вывод: классификатор показал более лучшие результаты.

Как будет меняться качество модели при уменьшении/увеличении размера P?

При увеличении Р метрики f1 и precision уменьшаются, а recall увеличивается.
При уменьшении Р все метрики уменьшаются.